In [38]:
from google.cloud import storage
import pandas as pd
import dask.dataframe as dd
import gcsfs
import io
import re
import warnings
warnings.filterwarnings("ignore")

In [39]:
storage_client = storage.Client()
bucket_name = "smtcmp-genai-data-hcb-dev"  # Replace with your bucket name
storage_bucket = storage_client.get_bucket(bucket_name)
fs = gcsfs.GCSFileSystem(project='anbc-hcb-dev')

## Readmission EDA

In [40]:
with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/Hospitals/Readmission_Deaths/04_2025_FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv') as f:
    readmission_df = pd.read_csv(f)

In [41]:
print("\n1. DATASET OVERVIEW:")
print(f"Total rows: {len(readmission_df):,}")
print(f"Total columns: {readmission_df.shape[1]}")
print(f"Dataset shape: {readmission_df.shape}")
print(f"Columns Name and Types: {readmission_df.dtypes}")


1. DATASET OVERVIEW:
Total rows: 18,510
Total columns: 12
Dataset shape: (18510, 12)
Columns Name and Types: Facility Name                  object
Facility ID                     int64
State                          object
Measure Name                   object
Number of Discharges          float64
Footnote                      float64
Excess Readmission Ratio      float64
Predicted Readmission Rate    float64
Expected Readmission Rate     float64
Number of Readmissions         object
Start Date                     object
End Date                       object
dtype: object


In [42]:
#How many unique hospitals

In [43]:
unquie_hospitals = readmission_df['Facility ID'].nunique()
print("Number of unique hospitals",unquie_hospitals)

Number of unique hospitals 3085


In [44]:
#What are the unique measure name values

In [45]:
print("n4. ALL READMISSION MEASURES")
unique_measures = readmission_df['Measure Name'].unique()
print(f"NUmber of unique measure: {len(unique_measures)}")
print("List of all measure:")
for i,measure in enumerate(sorted(unique_measures),1):
    measure_count = readmission_df[readmission_df['Measure Name']==measure].shape[0]
    print(f"{i:2d}. {measure} (n={measure_count:,} hospitals)")

n4. ALL READMISSION MEASURES
NUmber of unique measure: 6
List of all measure:
 1. READM-30-AMI-HRRP (n=3,085 hospitals)
 2. READM-30-CABG-HRRP (n=3,085 hospitals)
 3. READM-30-COPD-HRRP (n=3,085 hospitals)
 4. READM-30-HF-HRRP (n=3,085 hospitals)
 5. READM-30-HIP-KNEE-HRRP (n=3,085 hospitals)
 6. READM-30-PN-HRRP (n=3,085 hospitals)


In [46]:
#State Distribution
print(f"\n5 STATE DISTRIBUTION:")
state_counts = readmission_df['State'].value_counts()
print(f"NUmber of States: {len(state_counts)}")
print(f"Top 10 states by number of hospital-measure combinations: {state_counts.head(10)}")


5 STATE DISTRIBUTION:
NUmber of States: 51
Top 10 states by number of hospital-measure combinations: State
TX    1704
CA    1674
FL    1002
PA     804
NY     792
OH     714
IL     696
GA     576
MI     546
NC     492
Name: count, dtype: int64


In [47]:
#Missing Value Analysis

In [48]:
print(f"\n6 MISSING VALUE ANALYSIS:")
missing_summary = readmission_df.isnull().sum()
missing_percet = missing_summary/len(readmission_df) * 100
missing_df = pd.DataFrame({'Missing Count': missing_summary,'Missing Percent': missing_percet.round(2)})
print(missing_df[missing_df['Missing Count']>0])


6 MISSING VALUE ANALYSIS:
                            Missing Count  Missing Percent
Number of Discharges                10170            54.94
Footnote                            11927            64.44
Excess Readmission Ratio             6583            35.56
Predicted Readmission Rate           6583            35.56
Expected Readmission Rate            6583            35.56
Number of Readmissions               6583            35.56


In [49]:
# 7. Key numeric variables summary
print("\n7. KEY NUMERIC VARIABLES SUMMARY:")
numeric_cols = ['Number of Discharges', 'Excess Readmission Ratio', 
                'Predicted Readmission Rate', 'Expected Readmission Rate', 
                'Number of Readmissions']

for col in numeric_cols:
    if col in readmission_df.columns:
        # Convert to numeric, handling any non-numeric values
        readmission_df[col] = pd.to_numeric(readmission_df[col], errors='coerce')
        print(f"\n{col}:")
        print(f"  Mean: {readmission_df[col].mean():.4f}")
        print(f"  Median: {readmission_df[col].median():.4f}")
        print(f"  Min: {readmission_df[col].min():.4f}")
        print(f"  Max: {readmission_df[col].max():.4f}")
        print(f"  Missing: {readmission_df[col].isnull().sum()}")


7. KEY NUMERIC VARIABLES SUMMARY:

Number of Discharges:
  Mean: 279.2699
  Median: 197.0000
  Min: 0.0000
  Max: 4501.0000
  Missing: 10170

Excess Readmission Ratio:
  Mean: 1.0017
  Median: 0.9982
  Min: 0.4779
  Max: 1.6430
  Missing: 6583

Predicted Readmission Rate:
  Mean: 14.9954
  Median: 16.0602
  Min: 1.6742
  Max: 27.8095
  Missing: 6583

Expected Readmission Rate:
  Mean: 14.9612
  Median: 16.1460
  Min: 2.8921
  Max: 25.3942
  Missing: 6583

Number of Readmissions:
  Mean: 48.0749
  Median: 31.0000
  Min: 11.0000
  Max: 877.0000
  Missing: 10389


In [50]:
# 9. Check for hospitals with multiple measures
print("\n9. HOSPITAL MEASURE COVERAGE:")
hospital_measure_counts = readmission_df.groupby('Facility ID')['Measure Name'].count()
print(f"Average measures per hospital: {hospital_measure_counts.mean():.2f}")
print(f"Min measures per hospital: {hospital_measure_counts.min()}")
print(f"Max measures per hospital: {hospital_measure_counts.max()}")


9. HOSPITAL MEASURE COVERAGE:
Average measures per hospital: 6.00
Min measures per hospital: 6
Max measures per hospital: 6


In [51]:
# 10. Time period verification
print("\n10. TIME PERIOD VERIFICATION:")
print(f"Start dates: {readmission_df['Start Date'].unique()}")
print(f"End dates: {readmission_df['End Date'].unique()}")


10. TIME PERIOD VERIFICATION:
Start dates: ['07/01/2020']
End dates: ['06/30/2023']


In [52]:
# 11. Check for any footnote patterns
print("\n11. FOOTNOTE ANALYSIS:")
if 'Footnote' in readmission_df.columns:
    footnote_counts = readmission_df['Footnote'].value_counts()
    print("Footnote distribution:")
    print(footnote_counts.head(10))


11. FOOTNOTE ANALYSIS:
Footnote distribution:
Footnote
5.0    3272
1.0    3092
7.0     219
Name: count, dtype: int64


In [53]:
#Analyze missing data measure
# Analyze missing data by measure
missing_by_measure = readmission_df.groupby('Measure Name')['Excess Readmission Ratio'].agg(['count', 'size'])
missing_by_measure['missing'] = missing_by_measure['size'] - missing_by_measure['count']
missing_by_measure['missing_pct'] = (missing_by_measure['missing'] / missing_by_measure['size']) * 100
print("Missing data by measure:")
print(missing_by_measure.sort_values('missing_pct'))

Missing data by measure:
                        count  size  missing  missing_pct
Measure Name                                             
READM-30-PN-HRRP         2731  3085      354    11.474878
READM-30-HF-HRRP         2638  3085      447    14.489465
READM-30-COPD-HRRP       2324  3085      761    24.667747
READM-30-AMI-HRRP        1763  3085     1322    42.852512
READM-30-HIP-KNEE-HRRP   1588  3085     1497    48.525122
READM-30-CABG-HRRP        883  3085     2202    71.377634


In [54]:
readmission_df.to_csv('readmission_processed.csv', index=False)
#print(f"\nProcessed data saved to: county_health_combined_processed.csv")

## County Health Data

In [55]:
with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2020.csv') as f:
    countyh_df_20 = pd.read_csv(f)
# Use first row as column names
countyh_df_20 = countyh_df_20.drop(countyh_df_20.index[0]).reset_index(drop=True)


with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2021.csv') as f:
    countyh_df_21 = pd.read_csv(f)
# Use first row as column names
countyh_df_21 = countyh_df_21.drop(countyh_df_21.index[0]).reset_index(drop=True)


with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2022.csv') as f:
    countyh_df_22 = pd.read_csv(f)
# Use first row as column names
countyh_df_22 = countyh_df_22.drop(countyh_df_22.index[0]).reset_index(drop=True)    

with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2023.csv') as f:
    countyh_df_23 = pd.read_csv(f)
# Use first row as column names
countyh_df_23 = countyh_df_23.drop(countyh_df_23.index[0]).reset_index(drop=True)

with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2024.csv') as f:
    countyh_df_24 = pd.read_csv(f)
# Use first row as column names
countyh_df_24 = countyh_df_24.drop(countyh_df_24.index[0]).reset_index(drop=True)

with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/County-Health-Ranking/County_Health_Ranking_2025.csv') as f:
    countyh_df_25 = pd.read_csv(f)

# Use first row as column names
countyh_df_25 = countyh_df_25.drop(countyh_df_25.index[0]).reset_index(drop=True)

In [56]:
def clean_columns (df: pd.DataFrame):
    
    cleaned_columns = []
    for col in df.columns:
        if pd.isna(col):
            cleaned_columns.append('unnamed_column')
        else:
            clean_col = str(col).lower()
            clean_col = re.sub(r'[/\.&\-\s\(\)\,\\#\+\=\[\]]+', '_', clean_col)
            clean_col = re.sub(r'_+', '_', clean_col)
            clean_col = clean_col.strip('_')
            cleaned_columns.append(clean_col)

    df.columns = cleaned_columns
    return df

In [57]:
countyh_df_20_clean = clean_columns(countyh_df_20)
countyh_df_21_clean = clean_columns(countyh_df_21)
countyh_df_22_clean = clean_columns(countyh_df_22)
countyh_df_23_clean = clean_columns(countyh_df_23)
countyh_df_24_clean = clean_columns(countyh_df_24)
countyh_df_25_clean = clean_columns(countyh_df_25)

In [58]:
print(f"County Health 2020 shape: {countyh_df_20_clean.shape}")
print(f"County Health 2021 shape: {countyh_df_21_clean.shape}")
print(f"County Health 2022 shape: {countyh_df_22_clean.shape}")
print(f"County Health 2023 shape: {countyh_df_23_clean.shape}")
print(f"County Health 2024 shape: {countyh_df_24_clean.shape}")
print(f"County Health 2025 shape: {countyh_df_25_clean.shape}")

County Health 2020 shape: (3194, 786)
County Health 2021 shape: (3194, 690)
County Health 2022 shape: (3194, 725)
County Health 2023 shape: (3194, 720)
County Health 2024 shape: (3195, 770)
County Health 2025 shape: (3204, 796)


In [59]:
required_columns = ['State FIPS Code',
'County FIPS Code',
'5-digit FIPS Code',
'State Abbreviation',
'Name',
'Release Year',
'Median household income raw value',
'Median household income CI low',
'Median household income CI high',
'Children in poverty raw value',
'Children in poverty CI low',
'Children in poverty CI high',
'Unemployment raw value',
'Income inequality raw value',
'Some college raw value',
'High school graduation raw value',
'Uninsured adults raw value',
'Primary care physicians raw value',
'Ratio of population to primary care physicians.',
'Mental health providers raw value',
'Ratio of population to mental health providers.',
'Severe housing problems raw value',
'Social associations raw value',
'Residential segregation - Black/White raw value',
'% 65 and older raw value',
'% Non-Hispanic Black raw value',
'% American Indian & Alaska Native raw value',
'% Asian raw value',
'% Hispanic raw value',
'% Native Hawaiian/Other Pacific Islander raw value',
'% Non-Hispanic White raw value',
'% Rural raw value',
'% Females raw value',
'Population raw value',
'Adult smoking raw value',
'Adult obesity raw value',
'Excessive drinking raw value',
'Physical inactivity raw value',
'Poor or fair health raw value',
'Poor physical health days raw value',
'Poor mental health days raw value',
'Children in single-parent households raw value',
'Food insecurity raw value',
'Homeownership raw value',
'% not proficient in English raw value',
'Uninsured children raw value',
'Other primary care providers raw value']

In [60]:
required_columns_cleaned = []
for col in required_columns:
    clean_col = str(col).lower()
    clean_col = re.sub(r'[/\.&\-\s\(\)\,\\#\+\=\[\]]+', '_', clean_col)
    clean_col = re.sub(r'_+', '_', clean_col)
    clean_col = clean_col.strip('_')
    required_columns_cleaned.append(clean_col)

#required_columns_cleaned

In [61]:
# STEP 2: COMBINE ALL YEARS
countyH_df_all_cols = pd.concat([countyh_df_20_clean, countyh_df_21_clean,
                                 countyh_df_22_clean, countyh_df_23_clean, 
                                countyh_df_24_clean,
                                countyh_df_25_clean], ignore_index=True)
print(f"\nCombined data shape: {countyH_df_all_cols.shape}")


Combined data shape: (19175, 1018)


In [62]:
countyH_final_df = countyH_df_all_cols[required_columns_cleaned].copy()
print(f"\nFinal dataset shape: {countyH_final_df.shape}")


Final dataset shape: (19175, 47)


In [63]:
int_cols = ['state_fips_code','county_fips_code','5_digit_fips_code','release_year']
object_cols = ['name','state_abbreviation']
float_cols = list(countyH_final_df.columns.difference(int_cols+object_cols))
float_cols = [str(col) for col in float_cols]

#Convert the specified columns to float
for col in float_cols:
    try:
        countyH_final_df[col] = pd.to_numeric(countyH_final_df[col],errors='coerce')
    except Exception as e:
        print(f"Skipping {col} due to error {e}")

#Convert the specified columns to int
for col in int_cols:
    countyH_final_df[col] = pd.to_numeric(countyH_final_df[col],errors='coerce').astype('Int64')

#countyH_final_df['release_year'] = countyH_final_df['release_year'].astype(int)
#countyH_final_df['median_household_income_raw_value'] = countyH_final_df['median_household_income_raw_value'].astype(float)
#countyH_final_df['children_in_poverty_raw_value'] = countyH_final_df['children_in_poverty_raw_value'].astype(float)
#countyH_final_df['unemployment_raw_value'] = countyH_final_df['unemployment_raw_value'].astype(float)
#countyH_final_df['income_inequality_raw_value'] = countyH_final_df['income_inequality_raw_value'].astype(float)


## COUNTY HEALTH DATA QUALITY ANALYSIS

In [64]:
# Basic information
print(f"\n1. BASIC INFORMATION:")
print(f"   Shape: {countyH_final_df.shape}")
print(f"   Memory usage: {countyH_final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


1. BASIC INFORMATION:
   Shape: (19175, 47)
   Memory usage: 9.03 MB


In [65]:
# Year distribution
print(f"\n2. YEAR DISTRIBUTION:")
if 'release_year' in countyH_final_df.columns:
    year_counts = countyH_final_df['release_year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"   {year}: {count:,} counties")


2. YEAR DISTRIBUTION:
   2020: 3,194 counties
   2021: 3,194 counties
   2022: 3,194 counties
   2023: 3,194 counties
   2024: 3,195 counties
   2025: 3,204 counties


In [66]:
# Geographic coverage
print(f"\n3. GEOGRAPHIC COVERAGE:")
if 'state_abbreviation' in countyH_final_df.columns:
    print(f"   Unique states: {countyH_final_df['state_abbreviation'].nunique()}")
if 'name' in countyH_final_df.columns:
    print(f"   Unique counties: {countyH_final_df['name'].nunique()}")
if '5_digit_fips_code' in countyH_final_df.columns:
    print(f"   Unique FIPS codes: {countyH_final_df['5_digit_fips_code'].nunique()}")


3. GEOGRAPHIC COVERAGE:
   Unique states: 52
   Unique counties: 1940
   Unique FIPS codes: 3205


In [67]:
# Missing data analysis
print(f"\n4. MISSING DATA ANALYSIS:")
missing_data = countyH_final_df.isnull().sum()
missing_percent = (missing_data / len(countyH_final_df)) * 100

missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percent': missing_percent
}).sort_values('Missing_Percent', ascending=False)

# Show variables with missing data
variables_with_missing = missing_summary[missing_summary['Missing_Percent'] > 0]
print(f"   Variables with missing data: {len(variables_with_missing)}")
print("\n   Top 15 variables with most missing data:")
for idx, row in variables_with_missing.head(15).iterrows():
    print(f"   {idx}: {row['Missing_Count']:,} ({row['Missing_Percent']:.1f}%)")


4. MISSING DATA ANALYSIS:
   Variables with missing data: 41

   Top 15 variables with most missing data:
   %_females_raw_value: 12,787.0 (66.7%)
   %_native_hawaiian_other_pacific_islander_raw_value: 9,593.0 (50.0%)
   %_american_indian_alaska_native_raw_value: 9,593.0 (50.0%)
   residential_segregation_black_white_raw_value: 6,403.0 (33.4%)
   high_school_graduation_raw_value: 3,480.0 (18.1%)
   ratio_of_population_to_mental_health_providers: 1,220.0 (6.4%)
   mental_health_providers_raw_value: 1,220.0 (6.4%)
   ratio_of_population_to_primary_care_physicians: 920.0 (4.8%)
   primary_care_physicians_raw_value: 920.0 (4.8%)
   other_primary_care_providers_raw_value: 97.0 (0.5%)
   income_inequality_raw_value: 61.0 (0.3%)
   %_rural_raw_value: 36.0 (0.2%)
   median_household_income_ci_high: 16.0 (0.1%)
   median_household_income_ci_low: 16.0 (0.1%)
   median_household_income_raw_value: 16.0 (0.1%)


In [68]:

# Data completeness by year
print(f"\n5. DATA COMPLETENESS BY YEAR:")
if 'release_year' in countyH_final_df.columns:
    for year in sorted(countyH_final_df['release_year'].unique()):
        year_data = countyH_final_df[countyH_final_df['release_year'] == year]
        total_cells = year_data.shape[0] * year_data.shape[1]
        missing_cells = year_data.isnull().sum().sum()
        completeness = (1 - missing_cells / total_cells) * 100
        print(f"   {year}: {completeness:.1f}% complete ({missing_cells:,} missing cells)")


5. DATA COMPLETENESS BY YEAR:
   2020: 98.7% complete (1,965 missing cells)
   2021: 98.4% complete (2,385 missing cells)
   2022: 96.2% complete (5,674 missing cells)
   2023: 91.9% complete (12,223 missing cells)
   2024: 92.0% complete (12,051 missing cells)
   2025: 91.8% complete (12,352 missing cells)


In [69]:
# Key variables analysis
print(f"\n6. KEY VARIABLES ANALYSIS:")

# Economic variables
economic_vars = ['median_household_income_raw_value', 'children_in_poverty_raw_value', 'unemployment_raw_value', 
                 'income_inequality_raw_value']
print("\n   ECONOMIC VARIABLES:")
for var in economic_vars:
    if var in countyH_final_df.columns:
        missing_pct = (countyH_final_df[var].isnull().sum() / len(countyH_final_df)) * 100
        print(f"   {var}: {missing_pct:.1f}% missing")
        if countyH_final_df[var].dtype in ['int64', 'float64']:
            try:
                print(f"      Mean: {countyH_final_df[var].mean():.2f}, Std: {countyH_final_df[var].std():.2f}")
            except:
                print(f"      Cannot calculate statistics")


6. KEY VARIABLES ANALYSIS:

   ECONOMIC VARIABLES:
   median_household_income_raw_value: 0.1% missing
      Mean: 59124.48, Std: 15815.36
   children_in_poverty_raw_value: 0.1% missing
      Mean: 0.20, Std: 0.08
   unemployment_raw_value: 0.1% missing
      Mean: 0.04, Std: 0.02
   income_inequality_raw_value: 0.3% missing
      Mean: 4.53, Std: 0.79


In [70]:
# Healthcare variables
healthcare_vars = ['uninsured_adults_raw_value', 'primary_care_physicians_raw_value', 'mental_health_providers_raw_value']
print("\n   HEALTHCARE ACCESS VARIABLES:")
for var in healthcare_vars:
    if var in countyH_final_df.columns:
        missing_pct = (countyH_final_df[var].isnull().sum() / len(countyH_final_df)) * 100
        print(f"   {var}: {missing_pct:.1f}% missing")
        if countyH_final_df[var].dtype in ['int64', 'float64']:
            try:
                print(f"      Mean: {countyH_final_df[var].mean():.4f}, Std: {countyH_final_df[var].std():.4f}")
            except:
                print(f"      Cannot calculate statistics")


   HEALTHCARE ACCESS VARIABLES:
   uninsured_adults_raw_value: 0.1% missing
      Mean: 0.1346, Std: 0.0609
   primary_care_physicians_raw_value: 4.8% missing
      Mean: 0.0005, Std: 0.0004
   mental_health_providers_raw_value: 6.4% missing
      Mean: 0.0019, Std: 0.0021


In [71]:


# Education variables
education_vars = ['some_college_raw_value', 'high_school_graduation_raw_value']
print("\n   EDUCATION VARIABLES:")
for var in education_vars:
    if var in countyH_final_df.columns:
        missing_pct = (countyH_final_df[var].isnull().sum() / len(countyH_final_df)) * 100
        print(f"   {var}: {missing_pct:.1f}% missing")
        if countyH_final_df[var].dtype in ['int64', 'float64']:
            try:
                print(f"      Mean: {countyH_final_df[var].mean():.2f}, Std: {countyH_final_df[var].std():.2f}")
            except:
                print(f"      Cannot calculate statistics")


   EDUCATION VARIABLES:
   some_college_raw_value: 0.0% missing
      Mean: 0.59, Std: 0.12
   high_school_graduation_raw_value: 18.1% missing
      Mean: 0.88, Std: 0.07


In [72]:
countyH_final_df.dtypes

state_fips_code                                         Int64
county_fips_code                                        Int64
5_digit_fips_code                                       Int64
state_abbreviation                                     object
name                                                   object
release_year                                            Int64
median_household_income_raw_value                     float64
median_household_income_ci_low                        float64
median_household_income_ci_high                       float64
children_in_poverty_raw_value                         float64
children_in_poverty_ci_low                            float64
children_in_poverty_ci_high                           float64
unemployment_raw_value                                float64
income_inequality_raw_value                           float64
some_college_raw_value                                float64
high_school_graduation_raw_value                      float64
uninsure

In [73]:

# Demographics variables
demo_vars = ['%_65_and_older_raw_value', '%_non_hispanic_black_raw_value', '%_american_indian_alaska_native_raw_value', 
             '%_asian_raw_value','%_hispanic_raw_value','%_native_hawaiian_other_pacific_islander_raw_value',
            '%_non_hispanic_white_raw_value','%_rural_raw_value','%_females_raw_value']
print("\n   DEMOGRAPHIC VARIABLES:")
for var in demo_vars:
    if var in countyH_final_df.columns:
        missing_pct = (countyH_final_df[var].isnull().sum() / len(countyH_final_df)) * 100
        print(f"   {var}: {missing_pct:.1f}% missing")
        if countyH_final_df[var].dtype in ['int64', 'float64']:
            try:
                print(f"      Mean: {countyH_final_df[var].mean():.2f}, Std: {countyH_final_df[var].std():.2f}")
            except:
                print(f"      Cannot calculate statistics")


   DEMOGRAPHIC VARIABLES:
   %_65_and_older_raw_value: 0.0% missing
      Mean: 0.20, Std: 0.05
   %_non_hispanic_black_raw_value: 0.0% missing
      Mean: 0.09, Std: 0.14
   %_american_indian_alaska_native_raw_value: 50.0% missing
      Mean: 0.02, Std: 0.08
   %_asian_raw_value: 0.0% missing
      Mean: 0.02, Std: 0.03
   %_hispanic_raw_value: 0.0% missing
      Mean: 0.10, Std: 0.14
   %_native_hawaiian_other_pacific_islander_raw_value: 50.0% missing
      Mean: 0.00, Std: 0.01
   %_non_hispanic_white_raw_value: 0.0% missing
      Mean: 0.75, Std: 0.20
   %_rural_raw_value: 0.2% missing
      Mean: 0.60, Std: 0.32
   %_females_raw_value: 66.7% missing
      Mean: 0.50, Std: 0.02


In [74]:
print(f"\n7. SAMPLE FIRST 5 ROWS:")
print(countyH_final_df.head())

print(f"\n8. COLUMN NAMES:")
print("Available columns in final dataset:")
for i, col in enumerate(countyH_final_df.columns):
    print(f"   {i+1}. {col}")

# Save the processed data
#final_df.to_csv('county_health_combined_processed.csv', index=False)
#print(f"\nProcessed data saved to: county_health_combined_processed.csv")


7. SAMPLE FIRST 5 ROWS:
   state_fips_code  county_fips_code  5_digit_fips_code state_abbreviation  \
0                0                 0                  0                 US   
1                1                 0               1000                 AL   
2                1                 1               1001                 AL   
3                1                 3               1003                 AL   
4                1                 5               1005                 AL   

             name  release_year  median_household_income_raw_value  \
0   United States          2020                            61937.0   
1         Alabama          2020                            49881.0   
2  Autauga County          2020                            59338.0   
3  Baldwin County          2020                            57588.0   
4  Barbour County          2020                            34382.0   

   median_household_income_ci_low  median_household_income_ci_high  \
0              

In [75]:
# Save the processed data
countyH_final_df.to_csv('county_health_processed.csv', index=False)
print(f"\nProcessed data saved to: county_health_processed.csv")


Processed data saved to: county_health_processed.csv


## Hospital General Information

In [76]:
with fs.open('gs://smtcmp-genai-data-hcb-dev/HPD Analytics/Hospitals/Readmission_Deaths/07_2025_Hospital_General_Information.csv') as f:
    hospital_general_df = pd.read_csv(f)

In [77]:
hospital_general_df

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,...,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,1,8,2,NaN,8,8,NaN,12,10,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,8,1,NaN,8,8,NaN,12,12,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,Acute Care Hospitals,Proprietary,...,0,7,2,NaN,8,8,NaN,12,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,7,0,NaN,8,8,NaN,12,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,2,0,NaN,8,Not Available,5.0,12,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5379,670322,LEGENT SURGICAL HOSPITAL PLANO,4100 MAPLESHADE LANE,PLANO,TX,75075,COLLIN,(972) 265-1050,Acute Care Hospitals,Proprietary,...,Not Available,Not Available,Not Available,5.0,8,Not Available,5.0,12,Not Available,5.0
5380,670326,BRUSHY CREEK FAMILY HOSPITAL LLC,230 DEER RIDGE DR,ROUND ROCK,TX,78681,WILLIAMSON,(512) 766-1400,Acute Care Hospitals,Proprietary,...,Not Available,Not Available,Not Available,19.0,Not Available,Not Available,19.0,Not Available,Not Available,19.0
5381,670327,EXCEPTIONAL COMMUNITY HOSPITAL LUBBOCK,6401 SPUR 327,LUBBOCK,TX,79424,LUBBOCK,(469) 341-7800,Acute Care Hospitals,Proprietary,...,Not Available,Not Available,Not Available,19.0,Not Available,Not Available,19.0,Not Available,Not Available,19.0
5382,671300,GRAHAM REGIONAL MEDICAL CENTER,1301 MONTGOMERY ROAD,GRAHAM,TX,76450,YOUNG,(940) 549-3400,Critical Access Hospitals,Government - Hospital District or Authority,...,Not Available,Not Available,Not Available,5.0,Not Available,Not Available,5.0,Not Available,Not Available,5.0


In [78]:
print("\n1. HOSPITAL DATASET OVERVIEW:")
print(f"Total rows: {len(hospital_general_df):,}")
print(f"Total columns: {hospital_general_df.shape[1]}")
print(f"Dataset shape: {hospital_general_df.shape}")
print(f"Columns Name and Types: {hospital_general_df.dtypes}")


1. HOSPITAL DATASET OVERVIEW:
Total rows: 5,384
Total columns: 38
Dataset shape: (5384, 38)
Columns Name and Types: Facility ID                                          object
Facility Name                                        object
Address                                              object
City/Town                                            object
State                                                object
ZIP Code                                              int64
County/Parish                                        object
Telephone Number                                     object
Hospital Type                                        object
Hospital Ownership                                   object
Emergency Services                                   object
Meets criteria for birthing friendly designation     object
Hospital overall rating                              object
Hospital overall rating footnote                     object
MORT Group Measure Count                   

In [79]:
### 1.1 Missing Data Analysis
# Check missing data patterns
missing_data = hospital_general_df.isnull().sum()
missing_percentage = (missing_data / len(hospital_general_df)) * 100
print("Missing Data Summary:")
print(pd.DataFrame({'Missing_Count': missing_data, 'Missing_Percentage': missing_percentage}))

Missing Data Summary:
                                                  Missing_Count  \
Facility ID                                                   0   
Facility Name                                                 0   
Address                                                       0   
City/Town                                                     0   
State                                                         0   
ZIP Code                                                      0   
County/Parish                                                 0   
Telephone Number                                              0   
Hospital Type                                                 0   
Hospital Ownership                                            0   
Emergency Services                                            0   
Meets criteria for birthing friendly designation           3154   
Hospital overall rating                                       0   
Hospital overall rating footnote        

### 1.2 Key Variables Inspection
**Priority Variables for Analysis:**
- `Facility ID` (Primary key for merging with readmissions data)
- `County/Parish` (Critical for linking to county health rankings)
- `State` (Geographic validation)
- `Hospital Type` (Control variable)
- `Hospital Ownership` (Control variable)
- `Emergency Services` (Control variable)
- `Hospital overall rating` (Control variable)

In [80]:
### 1.3 Data Type Validation

# Check data types and unique values
for col in ['Facility ID', 'County/Parish', 'State', 'Hospital Type', 'Hospital Ownership']:
    print(f"\n{col}:")
    print(f"  Data type: {hospital_general_df[col].dtype}")
    print(f"  Unique values: {hospital_general_df[col].nunique()}")
    print(f"  Sample values: {hospital_general_df[col].value_counts().head(3)}")


Facility ID:
  Data type: object
  Unique values: 5384
  Sample values: Facility ID
671301    1
010001    1
010005    1
Name: count, dtype: int64

County/Parish:
  Data type: object
  Unique values: 1549
  Sample values: County/Parish
LOS ANGELES    88
COOK           60
JEFFERSON      57
Name: count, dtype: int64

State:
  Data type: object
  Unique values: 56
  Sample values: State
TX    459
CA    379
FL    219
Name: count, dtype: int64

Hospital Type:
  Data type: object
  Unique values: 6
  Sample values: Hospital Type
Acute Care Hospitals         3131
Critical Access Hospitals    1368
Psychiatric                   627
Name: count, dtype: int64

Hospital Ownership:
  Data type: object
  Unique values: 12
  Sample values: Hospital Ownership
Voluntary non-profit - Private                 2280
Proprietary                                    1060
Government - Hospital District or Authority     512
Name: count, dtype: int64


In [81]:
## Phase 2: Geographic Data Standardization

### 2.1 County Name Standardization
#####**Critical Step:** County names must match between datasets for successful merging.

# Standardize county names
def standardize_county_names(county_name):
    if pd.isna(county_name):
        return county_name
    
    # Convert to string and clean
    county_clean = str(county_name).strip().upper()
    
    # Remove common suffixes
    county_clean = county_clean.replace(' COUNTY', '')
    county_clean = county_clean.replace(' PARISH', '')
    county_clean = county_clean.replace(' BOROUGH', '')
    
    return county_clean

hospital_general_df['County_Standardized'] = hospital_general_df['County/Parish'].apply(standardize_county_names)

In [82]:
### 2.2 State Standardization

# Ensure state codes are consistent
print("State distribution:")
print(hospital_general_df['State'].value_counts())

# Check for any non-standard state codes
standard_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 
                   'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 
                   'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 
                   'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC']


State distribution:
State
TX    459
CA    379
FL    219
IL    195
OH    194
NY    190
PA    188
LA    156
IN    149
MI    146
GA    146
WI    142
MN    135
KS    135
OK    129
NC    123
MO    122
TN    122
IA    119
AZ    106
KY    102
AL    100
WA    100
MS     99
CO     97
VA     95
NE     92
AR     85
MA     82
NJ     78
SC     66
MT     63
SD     62
OR     62
PR     60
MD     56
WV     55
UT     52
ID     48
ND     47
NV     45
NM     44
ME     37
CT     37
WY     30
NH     28
AK     25
HI     24
VT     17
DE     13
RI     13
DC     10
VI      2
GU      2
AS      1
MP      1
Name: count, dtype: int64


In [83]:
### 2.3 ZIP Code Analysis

# Analyze ZIP codes for potential county mapping backup
print(f"ZIP Code range: {hospital_general_df['ZIP Code'].min()} - {hospital_general_df['ZIP Code'].max()}")
print(f"ZIP Code missing: {hospital_general_df['ZIP Code'].isnull().sum()}")


ZIP Code range: 603 - 99929
ZIP Code missing: 0


## Phase 3: Hospital Characteristics Processing

In [84]:


### 3.1 Hospital Type Categorization

# Analyze hospital types
print("Hospital Type Distribution:")
print(hospital_general_df['Hospital Type'].value_counts())

# Create simplified categories if needed
def categorize_hospital_type(hospital_type):
    if pd.isna(hospital_type):
        return 'Unknown'
    
    hospital_type = str(hospital_type).upper()
    
    if 'ACUTE' in hospital_type:
        return 'Acute Care'
    elif 'CRITICAL' in hospital_type:
        return 'Critical Access'
    elif 'SPECIALTY' in hospital_type:
        return 'Specialty'
    else:
        return 'Other'

hospital_general_df['Hospital_Type_Category'] = hospital_general_df['Hospital Type'].apply(categorize_hospital_type)

Hospital Type Distribution:
Hospital Type
Acute Care Hospitals                    3131
Critical Access Hospitals               1368
Psychiatric                              627
Acute Care - Veterans Administration     132
Childrens                                 94
Acute Care - Department of Defense        32
Name: count, dtype: int64


In [85]:
### 3.2 Hospital Ownership Processing

# Analyze ownership patterns
print("Hospital Ownership Distribution:")
print(hospital_general_df['Hospital Ownership'].value_counts())

# Create ownership categories
def categorize_ownership(ownership):
    if pd.isna(ownership):
        return 'Unknown'
    
    ownership = str(ownership).upper()
    
    if 'GOVERNMENT' in ownership or 'PUBLIC' in ownership:
        return 'Public'
    elif 'PROPRIETARY' in ownership or 'PRIVATE' in ownership:
        return 'Private'
    elif 'VOLUNTARY' in ownership or 'NON-PROFIT' in ownership:
        return 'Non-Profit'
    else:
        return 'Other'

hospital_general_df['Ownership_Category'] = hospital_general_df['Hospital Ownership'].apply(categorize_ownership)


Hospital Ownership Distribution:
Hospital Ownership
Voluntary non-profit - Private                 2280
Proprietary                                    1060
Government - Hospital District or Authority     512
Government - Local                              396
Voluntary non-profit - Other                    361
Voluntary non-profit - Church                   272
Government - State                              208
Veterans Health Administration                  132
Physician                                        74
Government - Federal                             42
Department of Defense                            32
Tribal                                           15
Name: count, dtype: int64


In [86]:
### 3.3 Emergency Services Processing

# Process emergency services
print("Emergency Services Distribution:")
print(hospital_general_df['Emergency Services'].value_counts())

# Convert to binary
hospital_general_df['Has_Emergency_Services'] = hospital_general_df['Emergency Services'].map({'Yes': 1, 'No': 0})

Emergency Services Distribution:
Emergency Services
Yes    4475
No      909
Name: count, dtype: int64


In [87]:

### 3.4 Hospital Rating Processing

# Analyze hospital ratings
print("Hospital Rating Distribution:")
print(hospital_general_df['Hospital overall rating'].value_counts())
# Convert to numeric (handle 'Not Available' cases)
def convert_rating(rating):
    if pd.isna(rating) or rating == 'Not Available':
        return None
    try:
        return int(rating)
    except:
        return None

hospital_general_df['Hospital_Rating_Numeric'] = hospital_general_df['Hospital overall rating'].apply(convert_rating)


Hospital Rating Distribution:
Hospital overall rating
Not Available    2572
3                 816
4                 758
2                 583
5                 380
1                 275
Name: count, dtype: int64


## Phase 4: Data Integration Preparation

In [88]:
### 4.1 Create Merge Keys

# Create standardized merge key for county health rankings
hospital_general_df['State_County_Key'] = hospital_general_df['State'] + '_' + hospital_general_df['County_Standardized']

# Verify Facility ID format for readmissions merge
print(f"Facility ID format check:")
print(f"  Length range: {hospital_general_df['Facility ID'].str.len().min()} - {hospital_general_df['Facility ID'].str.len().max()}")
print(f"  Sample IDs: {hospital_general_df['Facility ID'].head()}")

Facility ID format check:
  Length range: 6 - 6
  Sample IDs: 0    010001
1    010005
2    010006
3    010007
4    010008
Name: Facility ID, dtype: object


In [89]:
### 4.2 Geographic Coverage Analysis

# Analyze geographic coverage
print("Geographic Coverage Analysis:")
print(f"Total hospitals: {len(hospital_general_df)}")
print(f"Unique states: {hospital_general_df['State'].nunique()}")
print(f"Unique counties: {hospital_general_df['County_Standardized'].nunique()}")

# State-level distribution
state_counts = hospital_general_df['State'].value_counts()
print(f"\nTop 10 states by hospital count:")
print(state_counts.head(10))

Geographic Coverage Analysis:
Total hospitals: 5384
Unique states: 56
Unique counties: 1549

Top 10 states by hospital count:
State
TX    459
CA    379
FL    219
IL    195
OH    194
NY    190
PA    188
LA    156
IN    149
MI    146
Name: count, dtype: int64


In [90]:

### 4.3 Data Completeness Assessment

# Assess completeness of key variables
key_vars = ['Facility ID', 'County_Standardized', 'State', 'Hospital_Type_Category', 
           'Ownership_Category', 'Has_Emergency_Services']

completeness = {}
for var in key_vars:
    completeness[var] = {
        'Complete': hospital_general_df[var].notna().sum(),
        'Missing': hospital_general_df[var].isna().sum(),
        'Completeness_Rate': (hospital_general_df[var].notna().sum() / len(hospital_general_df)) * 100
    }

completeness_df = pd.DataFrame(completeness).T
print("Data Completeness for Key Variables:")
print(completeness_df)

Data Completeness for Key Variables:
                        Complete  Missing  Completeness_Rate
Facility ID               5384.0      0.0              100.0
County_Standardized       5384.0      0.0              100.0
State                     5384.0      0.0              100.0
Hospital_Type_Category    5384.0      0.0              100.0
Ownership_Category        5384.0      0.0              100.0
Has_Emergency_Services    5384.0      0.0              100.0


## Phase 5: Quality Checks & Validation

In [91]:
### 5.1 Duplicate Detection

# Check for duplicate facilities
duplicates = hospital_general_df[hospital_general_df.duplicated(subset=['Facility ID'], keep=False)]
print(f"Duplicate Facility IDs: {len(duplicates)}")

if len(duplicates) > 0:
    print("Duplicate facilities found:")
    print(duplicates[['Facility ID', 'Facility Name', 'State', 'County_Standardized']])

Duplicate Facility IDs: 0


In [92]:
### 5.2 Geographic Validation

# Cross-validate state and county combinations
state_county_combos = hospital_general_df.groupby(['State', 'County_Standardized']).size().reset_index(name='Hospital_Count')
print(f"Unique state-county combinations: {len(state_county_combos)}")

# Look for potential data quality issues
suspicious_combos = state_county_combos[state_county_combos['Hospital_Count'] > 20]
print(f"Counties with >20 hospitals (potential data quality check needed):")
print(suspicious_combos)

Unique state-county combinations: 2451
Counties with >20 hospitals (potential data quality check needed):
     State County_Standardized  Hospital_Count
134     AZ            MARICOPA              53
159     CA         LOS ANGELES              88
170     CA              ORANGE              28
175     CA      SAN BERNARDINO              23
176     CA           SAN DIEGO              22
547     IL                COOK              58
1032    MI               WAYNE              21
1529    NV               CLARK              24
1722    OK            OKLAHOMA              23
1820    PA        PHILADELPHIA              21
2052    TX              DALLAS              30
2083    TX              HARRIS              49
2158    TX             TARRANT              29


## Phase 6: Final Dataset Creation

In [93]:


### 6.1 Create Analysis-Ready Dataset
# Select final variables for analysis
analysis_vars = [
    'Facility ID',                    # Primary key
    'Facility Name',                  # Reference
    'State',                         # Geographic
    'County_Standardized',           # Geographic (for county health rankings merge)
    'State_County_Key',              # Merge key
    'Hospital_Type_Category',        # Control variable
    'Ownership_Category',            # Control variable
    'Has_Emergency_Services',        # Control variable
    'Hospital_Rating_Numeric',       # Control variable
    'ZIP Code'                       # Backup geographic identifier
]

hospital_analysis_df = hospital_general_df[analysis_vars].copy()


### 6.2 Export for Integration

In [94]:
# Save preprocessed dataset
hospital_analysis_df.to_csv('hospital_general_preprocessed.csv', index=False)

# Create summary statistics
summary_stats = hospital_analysis_df.describe(include='all')
print("Final Dataset Summary:")
print(summary_stats)


Final Dataset Summary:
       Facility ID      Facility Name State County_Standardized  \
count         5384               5384  5384                5384   
unique        5384               5257    56                1549   
top         671301  MEMORIAL HOSPITAL    TX         LOS ANGELES   
freq             1                 12   459                  88   
mean           NaN                NaN   NaN                 NaN   
std            NaN                NaN   NaN                 NaN   
min            NaN                NaN   NaN                 NaN   
25%            NaN                NaN   NaN                 NaN   
50%            NaN                NaN   NaN                 NaN   
75%            NaN                NaN   NaN                 NaN   
max            NaN                NaN   NaN                 NaN   

       State_County_Key Hospital_Type_Category Ownership_Category  \
count              5384                   5384               5384   
unique             2451           

## THREE WAY DATA MERGE

In [95]:
from typing import Tuple, Dict, List

def standardize_county_names(county_series: pd.Series) -> pd.Series:
    """
    Standardize county names by removing common suffixes and cleaning formatting
    """
    # Convert to string and handle missing values
    county_clean = county_series.astype(str).str.strip()
    
    # Remove common county suffixes
    suffixes_to_remove = [
        r'\s+County$', r'\s+Parish$', r'\s+Borough$', 
        r'\s+city$', r'\s+City$', r'\s+COUNTY$', 
        r'\s+PARISH$', r'\s+BOROUGH$'
    ]
    
    for suffix in suffixes_to_remove:
        county_clean = county_clean.str.replace(suffix, '', regex=True)
    
    # Additional cleaning
    county_clean = county_clean.str.strip()
    county_clean = county_clean.str.title()  # Standardize capitalization
    
    return county_clean

def load_and_prepare_hospital_general(file_path: str) -> pd.DataFrame:
    """
    Load and prepare Hospital General Information dataset
    """
    print("Loading Hospital General Information dataset...")
    
    # Load the dataset
    hospital_general = pd.read_csv(file_path)
    
    
    # Keep only existing columns
    existing_columns = [col for col in hospital_general.columns]
    hospital_general = hospital_general[existing_columns].copy()
    
    # Clean and standardize
    hospital_general['county_clean'] = standardize_county_names(hospital_general['County_Standardized'])
    hospital_general['state_county'] = hospital_general['State'].astype(str) + '_' + hospital_general['county_clean']
    
    # Clean Facility ID to ensure 6-character format
    hospital_general['Facility ID'] = hospital_general['Facility ID'].astype(str).str.strip()
    
    # Remove duplicates based on Facility ID
    hospital_general = hospital_general.drop_duplicates(subset=['Facility ID'])
    
    print(f"Hospital General dataset prepared: {len(hospital_general)} hospitals")
    print(f"Geographic coverage: {hospital_general['State'].nunique()} states, {hospital_general['county_clean'].nunique()} counties")
    
    return hospital_general

def load_and_prepare_county_health(file_path: str) -> pd.DataFrame:
    """
    Load and prepare County Health Rankings dataset
    """
    print("Loading County Health Rankings dataset...")
    
    # Load the dataset
    county_health = pd.read_csv(file_path)
    
    # Filter to 2020-2022 to match readmissions period
    county_health = county_health[county_health['release_year'].isin([2020, 2021, 2022])].copy()
    """
    # Select core social determinant variables with low missing data
    core_variables = [
        'release_year',
        'state_abbreviation',
        'name',
        '5_digit_fips_code',
        # Economic factors
        'median_household_income_raw_value',
        'children_in_poverty_raw_value',
        'unemployment_raw_value',
        # Healthcare access
        'uninsured_adults_raw_value',
        'primary_care_physicians_raw_value',
        'mental_health_providers_raw_value',
        # Demographics
       '%_65_and_older_raw_value',
       '%_non_hispanic_black_raw_value',
       '%_american_indian_alaska_native_raw_value',
       '%_asian_raw_value',
       '%_hispanic_raw_value',
       '%_native_hawaiian_other_pacific_islander_raw_value',
       '%_non_hispanic_white_raw_value',
       '%_rural_raw_value',
       '%_females_raw_value'
        # Education
        'some_college_raw_value'
    ]
    """
    # Keep only existing columns
    existing_columns = [col for col in county_health.columns]
    county_health = county_health[existing_columns].copy()
    
    # Clean and standardize county names
    county_health['county_clean'] = standardize_county_names(county_health['name'])
    county_health['state_county'] = county_health['state_abbreviation'].astype(str) + '_' + county_health['county_clean']
    
    # For multiple years, take the most recent available data (2022 first, then 2021, then 2020)
    # This handles the decreasing completeness over time
    county_health = county_health.sort_values(['state_county', 'release_year'], ascending=[True, False])
    county_health = county_health.groupby('state_county').first().reset_index()
    
    print(f"County Health dataset prepared: {len(county_health)} counties")
    print(f"Geographic coverage: {county_health['state_abbreviation'].nunique()} states")
    
    return county_health

def load_and_prepare_readmissions(file_path: str) -> pd.DataFrame:
    """
    Load and prepare Hospital Readmissions dataset
    """
    print("Loading Hospital Readmissions dataset...")
    
    # Load the dataset
    readmissions = pd.read_csv(file_path)
    
    # Filter to pneumonia readmissions (primary outcome)
    readmissions = readmissions[readmissions['Measure Name'] == 'READM-30-PN-HRRP'].copy()
    #readmissions = readmissions[readmissions['Measure Name'] == 'READM-30-HIP-KNEE-HRRP'].copy()
    """
    # Select relevant columns
    columns_to_keep = [
        'Facility ID',
        'Facility Name',
        'Measure ID',
        'Number of Discharges',
        'Footnote',
        'Excess Readmission Ratio',
        'Predicted Readmission Rate',
        'Expected Readmission Rate',
        'Number of Readmissions'
    ]
    """
    # Keep only existing columns
    existing_columns = [col for col in readmissions.columns]
    readmissions = readmissions[existing_columns].copy()
    
    # Clean Facility ID
    readmissions['Facility ID'] = readmissions['Facility ID'].astype(str).str.strip()
    
    # Remove records with missing outcome data
    readmissions = readmissions.dropna(subset=['Excess Readmission Ratio'])
    
    # Remove duplicates
    readmissions = readmissions.drop_duplicates(subset=['Facility ID'])
    
    print(f"Readmissions dataset prepared: {len(readmissions)} hospitals with pneumonia data")
    
    return readmissions

def execute_three_way_merge(hospital_general: pd.DataFrame, 
                           county_health: pd.DataFrame, 
                           readmissions: pd.DataFrame) -> Tuple[pd.DataFrame, Dict]:
    """
    Execute the three-way merge and return merged dataset with statistics
    """
    print("\n=== EXECUTING THREE-WAY MERGE ===")
    
    # Step 1: Merge Hospital General with County Health Rankings
    print("Step 1: Merging Hospital General with County Health Rankings...")
    
    merge_stats = {}
    
    # First merge on State_County
    hospital_county_merge = pd.merge(
        hospital_general, 
        county_health, 
        on='state_county', 
        how='left',
        suffixes=('_hospital', '_county')
    )
    
    # Check merge success
    successful_county_merge = hospital_county_merge['release_year'].notna().sum()
    merge_stats['hospital_county_successful'] = successful_county_merge
    merge_stats['hospital_county_total'] = len(hospital_general)
    merge_stats['hospital_county_rate'] = successful_county_merge / len(hospital_general) * 100
    
    print(f"Hospital-County merge: {successful_county_merge}/{len(hospital_general)} hospitals matched ({merge_stats['hospital_county_rate']:.1f}%)")
    
    # Step 2: Merge with Readmissions data
    print("Step 2: Merging with Readmissions data...")
    
    final_merged = pd.merge(
        hospital_county_merge,
        readmissions,
        on='Facility ID',
        how='inner',  # Only keep hospitals with readmissions data
        suffixes=('', '_readmissions')
    )
    
    # Check final merge success
    merge_stats['final_sample_size'] = len(final_merged)
    merge_stats['readmissions_total'] = len(readmissions)
    merge_stats['three_way_success_rate'] = len(final_merged) / len(readmissions) * 100
    
    print(f"Final three-way merge: {len(final_merged)} hospitals with complete data")
    print(f"Success rate from readmissions sample: {merge_stats['three_way_success_rate']:.1f}%")
    
    # Step 3: Clean final dataset
    print("Step 3: Cleaning final dataset...")
    
    # Remove duplicate columns and clean names
    final_merged = final_merged.loc[:, ~final_merged.columns.duplicated()]
    
    # Create final analysis variables
    #final_merged['Has_Emergency_Services'] = final_merged['Emergency Services'].map({'Yes': 1, 'No': 0})
    
    # Convert numeric columns
    numeric_columns = [
        'median_household_income_raw_value', 'children_in_poverty_raw_value', 'unemployment_raw_value',
        'uninsured_adults_raw_value', 'primary_care_physicians_raw_value',
        'mental_health_providers_raw_value',
           '%_65_and_older_raw_value',
           '%_non_hispanic_black_raw_value',
           '%_american_indian_alaska_native_raw_value',
           '%_asian_raw_value',
           '%_hispanic_raw_value',
           '%_native_hawaiian_other_pacific_islander_raw_value',
           '%_non_hispanic_white_raw_value',
           '%_rural_raw_value',
           '%_females_raw_value','Excess Readmission Ratio'
    ]
    
    for col in numeric_columns:
        if col in final_merged.columns:
            final_merged[col] = pd.to_numeric(final_merged[col], errors='coerce')
    
    # Calculate data completeness for key variables
    merge_stats['key_variable_completeness'] = {}
    for col in numeric_columns:
        if col in final_merged.columns:
            completeness = (1 - final_merged[col].isna().sum() / len(final_merged)) * 100
            merge_stats['key_variable_completeness'][col] = completeness
    
    print(f"Final dataset ready: {len(final_merged)} hospitals")
    
    return final_merged, merge_stats

def analyze_merge_results(merged_data: pd.DataFrame, merge_stats: Dict) -> None:
    """
    Analyze and display merge results
    """
    print("\n=== MERGE ANALYSIS RESULTS ===")
    
    print(f"\nFinal Sample Size: {merge_stats['final_sample_size']} hospitals")
    print(f"Geographic Coverage: {merged_data['State'].nunique()} states, {merged_data['state_county'].nunique()} counties")
    
    print(f"\nMerge Success Rates:")
    print(f"  Hospital-County merge: {merge_stats['hospital_county_rate']:.1f}%")
    print(f"  Three-way merge success: {merge_stats['three_way_success_rate']:.1f}%")
    
    print(f"\nHospital Characteristics Distribution:")
    print(f"  Hospital Type:\n{merged_data['Hospital_Type_Category'].value_counts()}")
    print(f"  Hospital Ownership:\n{merged_data['Ownership_Category'].value_counts()}")
    print(f"  Emergency Services: {merged_data['Has_Emergency_Services'].mean()*100:.1f}% have emergency services")
    
    print(f"\nKey Variable Completeness:")
    for var, completeness in merge_stats['key_variable_completeness'].items():
        print(f"  {var}: {completeness:.1f}%")
    
    print(f"\nPrimary Outcome (Excess Readmission Ratio):")
    err_stats = merged_data['Excess Readmission Ratio'].describe()
    print(f"  Mean: {err_stats['mean']:.3f}")
    print(f"  Std: {err_stats['std']:.3f}")
    print(f"  Range: {err_stats['min']:.3f} - {err_stats['max']:.3f}")



In [96]:
# Main execution function
def main():
    """
    Main function to execute the three-way merge
    """
    print("=== SOCIAL DETERMINANTS ANALYSIS: THREE-WAY DATA MERGE ===\n")
    
    # File paths - UPDATE THESE WITH YOUR ACTUAL FILE PATHS
    hospital_general_path = "hospital_general_preprocessed.csv"  # Update this path
    county_health_path = "county_health_processed.csv"  # Update this path
    readmissions_path = "readmission_processed.csv"  # Update this path
    
    try:
        # Load and prepare datasets
        hospital_general = load_and_prepare_hospital_general(hospital_general_path)
        county_health = load_and_prepare_county_health(county_health_path)
        readmissions = load_and_prepare_readmissions(readmissions_path)
        
        # Execute three-way merge
        merged_data, merge_stats = execute_three_way_merge(hospital_general, county_health, readmissions)
        
        # Analyze results
        analyze_merge_results(merged_data, merge_stats)
        
        # Save merged dataset
        output_path = "merged_social_determinants_analysis.csv"
        merged_data.to_csv(output_path, index=False)
        print(f"\nMerged dataset saved to: {output_path}")
        
        # Return for further analysis
        return merged_data, merge_stats
        
    except FileNotFoundError as e:
        print(f"Error: Could not find file. Please update the file paths in the main() function.")
        print(f"Missing file: {e}")
        return None, None
    except Exception as e:
        print(f"Error during merge process: {e}")
        return None, None

if __name__ == "__main__":
    merged_data, merge_stats = main()

=== SOCIAL DETERMINANTS ANALYSIS: THREE-WAY DATA MERGE ===

Loading Hospital General Information dataset...
Hospital General dataset prepared: 5384 hospitals
Geographic coverage: 56 states, 1549 counties
Loading County Health Rankings dataset...
County Health dataset prepared: 3180 counties
Geographic coverage: 52 states
Loading Hospital Readmissions dataset...
Readmissions dataset prepared: 2731 hospitals with pneumonia data

=== EXECUTING THREE-WAY MERGE ===
Step 1: Merging Hospital General with County Health Rankings...
Hospital-County merge: 5163/5384 hospitals matched (95.9%)
Step 2: Merging with Readmissions data...
Final three-way merge: 2237 hospitals with complete data
Success rate from readmissions sample: 81.9%
Step 3: Cleaning final dataset...
Final dataset ready: 2237 hospitals

=== MERGE ANALYSIS RESULTS ===

Final Sample Size: 2237 hospitals
Geographic Coverage: 42 states, 1208 counties

Merge Success Rates:
  Hospital-County merge: 95.9%
  Three-way merge success: 81.9%

In [97]:
merged_data.columns

Index(['Facility ID', 'Facility Name', 'State', 'County_Standardized',
       'State_County_Key', 'Hospital_Type_Category', 'Ownership_Category',
       'Has_Emergency_Services', 'Hospital_Rating_Numeric', 'ZIP Code',
       'county_clean_hospital', 'state_county', 'state_fips_code',
       'county_fips_code', '5_digit_fips_code', 'state_abbreviation', 'name',
       'release_year', 'median_household_income_raw_value',
       'median_household_income_ci_low', 'median_household_income_ci_high',
       'children_in_poverty_raw_value', 'children_in_poverty_ci_low',
       'children_in_poverty_ci_high', 'unemployment_raw_value',
       'income_inequality_raw_value', 'some_college_raw_value',
       'high_school_graduation_raw_value', 'uninsured_adults_raw_value',
       'primary_care_physicians_raw_value',
       'ratio_of_population_to_primary_care_physicians',
       'mental_health_providers_raw_value',
       'ratio_of_population_to_mental_health_providers',
       'severe_housing_probl